# Applying a Machine Learning to Record Linkage to Develop a Retail Arbitrage Procurement Solution
CPS840  
Josh Lindsay, 5008585364  
Dec 9, 2018

## Problem Summary:
Arbitrage is, "the simultaneous buying and selling of securities, currency, or commodities in different markets or in derivative forms in order to take advantage of differing prices for the same asset."

Simply put, profit(x) = selling_price(x) - purchase_price(x)

x can be any good which can be purchased or sold. An arbitrage 'opportunity' is an item, x, which can be purchased for less than it costs. This is most often applied to stocks or cryptocurrencies in the realm of computer science, since they can be easily bought, sold, and quantified, but arbitrage can be applied to any good. Selecting the good can be difficult because if the niche of the good is too small then the possible items to buy and sell are insignificiant. If the the niche of the good is too large, then the competition of other arbitrage systems will compete over opportunities. There are even some cases with stocks, where systems are created to take advantage of simpler exisisting systems!

In this case, arbitrage will be applied to consumer goods, specifically used electronics. Electronics can be easily sold since they are in large demand yet they also provide many arbitrage opprunities because people often miss-price items when selling items second hand. Since the selling process is time consuming (takes more effort to list an item on eBay than a stock exchange) there will be few if any competitors.

Since it is decided that x should be used electronics, we need to collect data for as many purchasable used electronics as possible. The solution to this problem will represent the data set, a set of x's, is represented in the "Data Collection" section.

Once the data set is collected, we have the purchase_price(x) component of our arbitrage equation. The machine learning component will be used to find the selling price. Then, the solution can just filter for products with an estimated selling price more than their purchase price. Most machine learning algorithms would use regression to predict what the selling would be (predictive analysis) using various metadata. However, given the limited percentage of arbitrage opportunities, using predictive analysis would be too error prone. Say 2% of x's can be resold at a profit, but the algorithm has an error rate of 4% (which is a generously low estimation) then it would be hard to differentiate between the actual opportunities and errors. 

Instead of estimating the price of the product, one could match the product with another similar/identical product (which has a known accurate price). This would be done with a classification algorithm could be used to match the found x with a dataset of accurate price. This essentially a problem of linking one dataset, A, to another, B. A is the dataset of items that could be purchased, and B represents a dataset of all possible items with their fair market price. If x from A (with purchase price p1) is matched to item y from B (with market price of p2), and p2 > p1, then the item represents an arbitrage opportunity! Our B data set is also obtained in the Data Collection section.

"Record linkage (RL), also referred to as data matching or entity resolution, is a process of finding records that correspond to the same entity from one or more data source." Linking the record from A to B is critical to solve the selling_price(x) problem. This field of study first noted by Halbert L. Dunn in a 1946 issue of The American Journal of Public Health. There are a variety of simple solutions from using the Levenshtein Distance algorithm to using custom software packages like Python Record Linkage Toolkit or ensemble ML algorithms.

An unsupervised ML algorithm will be used for linkage, treating it as a classification problem. The specific algorithm will be decided based on a tradeoff of speed and accuracy. Since large datasets need to be rechecked often, speed important. Since any opportunities will be validated by an individual, accuracy is less important. There isn't any novel ML developments in this solution, only a (hopefully) practical ML solution to a niche problem. This portion is solved in the 'Record Linkage with ML' section of the report. 

The ML solution will be compared with the using an implentation of Levenshtein Distance for a good baseline of effectiveness. They will be compared in terms of relative time and accuracy. Finally, some identified 'opportunities' will be investigated to validate the solution methadology. This process is in the 'Efficacy Analysis' portion of the report.

### Note:
This Jupyter notebook is intended to accompany the paper of the same name. It contains the methadology, code, problem analysis, summary of related work, etc. It is intended to be a standalone document but also serves as a reference for the code and proof of concept for the paper. Due to dependencies, the code for the data collection should not be run in the Jupyter notebook. However, the code could be copied and run on a local machine. Otherwise, the machine learning portion of this notebook can be run.

## Data Collection:
The goal is to find used electronics for sale which can be resold for a profit. Ideally, this would be applied to all consumer goods (and it can be), but in this case will be applied to specifically to used electronics listed on Kijiji and Craigslist in Canadian cities. The data is obtained the a process called 'scraping'. This is done by downloading large amounts of data to the web, parsing it to select certain data, and saving the parsed data. This can be done with a python library called 'Scrapy', "an open source and collaborative framework for extracting the data you need from websites. In a fast, simple, yet extensible way." 

Scrapy is a powerful framework which implements a variety of web related python frameworks. The problem and specific code can vary for the same web scraping process. The core logic of the frameworks for the scrapers is based off the code found here, https://www.digitalocean.com/community/tutorials/how-to-crawl-a-web-page-with-scrapy-and-python-3. 

The scraper or 'spider' works by parsing a given web page using html and css tags. What is parsed, how, and why varies for the data source. However, there is some consistency for parsing multi-page list data. 
1. An initial web page is given to the spider. 
2. Sections of web page's source code is seperated by a recurring tag, with each tag (hopefully) representing an item. 
3. Each item's block of source cose of code are then parsed for the desired data.
4. Items are saved in some format.
5. The next web page is loaded (by selecting some link on the page, in this case the 'next' button to load the next list page), and the process is repeated until no more web pages are loaded.

Three spiders will be created. Two will be to scrape Kijiji and Craigslist. The output of these two spiders represent the A datasource. The set of items that could be purchased and resold. Craigslist and Kijiji are two commonly used web marketplaces so they are a good dataset. Alternatives like eBay or Facebook marketplace are good alternatives by pose more complexity for web scraping. Although the the scraper be used to parse the description and images of the listings, this will not be done to reduce the time and complexity. If this proof of concept is successful, further investigation into a more advanced scraped dataset would be worthwhile.

The third spider will be used to scrape Amazon. The output of thus spider is the B datasource, the fair market price of all possible items. Amazon has recently beaten eBay as the best all-in-one source (and thus datasource) for consumer products. Although it is possible to parse all of Amazon, that represents an uncessarily large and time consuming dataset. If the dataset represented all items, an initial filter by category would be applied to false positives. So, a model is trained for just one category it is a fair representation for what a complete model would be like. Once again, if proved effective scraping all of Amazon would be worthwhile. 

To extend categories or cities, one must simple change the provided links to the scrapers (to be explained later). To add data sources, all one needs to do is to add a scraper(s) for said data source to collect and clean the data to match the same format of the Kijiji and Craigslist scrapers. 

## Creating the Amazon Spider
Since the Amazon spider only need to scrape once to create the B dataset, using proper coding practices is less important.

The average amazon search page has proximately 5000 lines of code. Trying to make sense of all of these lines of code is unecessary, since only information related to the product is required. Searching for the text of product name in the page source code returns the following information. It is one (very long) line of code, with each line similar to this style seperated by a line of white space, so each of these lines code must likely represents a block of code related to the item. The entire page source code is in the accompanying file, "Amazon Source HTML.htm".

A unique tag is need to seperate each of these items in our spider. Taking a quick glance, it a customer review a tag, a list tag, then a large amount of various divs with formatting. Since the formatting changes, it is not ideal to select these divs. However, 'div class="s-item-container"' seems to only repeat once and be at the start of the the block, so it is a a good tag to seperate on.

Now that the relevant code can be seperated by the item, a series of tags needs to be chosen to extract the relevant information of name, price, and address. 
1. Price is simple, it is contained in the unique 'span class="sx-price-whole"' span.
2. The product name is found in the 'data-attribute=' component of an h2 tag. Since there is only one h2 tag, this tag can be consistently used.
3. The address field is slightly more difficult. The location of the address varies depending on whether the item is displayed as in a different format like a bestseller, or as a standard item. If a standard item the address can be found in the href of the 'a' first tag. If it is in a different format, the first 'a' tag will contain the item id in the 'id=' portion of the 'a' tag. Luckily, a working item address can be created appending /dp/id to the the amazon domain.
4. Item category, which is in the title of the page.

There also need to be a way to pass the next page to be scraped to the spider. The next page of the search lisitings will be passed. The address for this is stored in the 'href' after the 'a' tag with the title 'Next Page'.

The actual selection of the tags described below are done with XPath. Explaining the XPath syntax is out of scope for this report. Scrapy automatically saves each item as a new line item in whatever format you specify, with the column values from the yield function. 

Category Location:
![Category:](images/amazon_category.png)
Item Location:
![Items:](images/amazon_items.png)
Next Page Location:
![Next Page:](images/amazon_next_page.png)

Here is the Amazon spider code:

In [1]:
# scrapy related imports
import scrapy
from scrapy import Request
from scrapy.crawler import CrawlerProcess
# system related imports
import sys
import time


# custom spider class which inherits the process of the master scrapy Spider class. 
class AmazonSpider(scrapy.Spider):
    # name, permitted domains, and a hardcoded start url
    name = "Amazon"
    allowed_domains = ["amazon.com"]
    start_urls = ["https://www.amazon.com/gp/search/ref=sr_nr_p_36_0?rnid=386442011&rh=n%3A172282%2Cn%3A%21493964%2Cn%3A502394%2Cp_n_condition-type%3A2224373011%2Cp_36%3A10000-99999999%2Cn%3A281052&qid=1544518365&bbn=281052&low-price=50&high-price=",
    "https://www.amazon.com/s/ref=sr_nr_n_4?fst=as%3Aoff&rh=n%3A172282%2Cn%3A%21493964%2Cn%3A502394%2Cp_n_condition-type%3A2224373011%2Cp_36%3A10000-99999999%2Cn%3A7161070011&bbn=502394&ie=UTF8&qid=1544518525&rnid=502394",
    "https://www.amazon.com/gp/search/ref=sr_nr_n_6?fst=as%3Aoff&rh=n%3A172282%2Cn%3A%21493964%2Cn%3A502394%2Cp_n_condition-type%3A2224373011%2Cp_36%3A10000-99999999%2Cn%3A499248&bbn=502394&ie=UTF8&qid=1544518525&rnid=502394",
    "https://www.amazon.com/s/ref=sr_nr_n_8?fst=as%3Aoff&rh=n%3A172282%2Cn%3A%21493964%2Cn%3A502394%2Cp_n_condition-type%3A2224373011%2Cp_36%3A10000-99999999%2Cn%3A7161073011&bbn=502394&ie=UTF8&qid=1544518568&rnid=502394",
    "https://www.amazon.com/s/ref=sr_st?rh=n%3A172282%2Cn%3A%21493964%2Cn%3A502394%2Cp_n_condition-type%3A2224373011%2Cp_36%3A10000-99999999&qid=1544518630&bbn=502394&sort=review-rank"]

    # parsing function to seperate the response (raw page code) and return desired items
    def parse(self, response):
        # function to check which kind of address tag is appropriate and return address
        def check_address(listing, s1, s2):
            a = listing.xpath(s1).extract_first().strip()
            if a[0] == '/': 
                i = listing.xpath(s2).extract_first().strip()
                return 'https://www.amazon.com/dp/' + i[i.find('_')+1:]
            else:
                return a

        # get the category from the page title
        category = response.xpath('.//title/text()').extract_first().strip()
        category = category[category.index('/ ')+2:]
        # selector to split the page into blocks of code for each item
        info_selector = '.s-item-container'
        # get timestamp
        date = time.strftime('%Y-%m-%d')

        # seperate and loop through page based on item selector
        for listings in response.css(info_selector):
            try:
                # select price as the text beside the sx-price-whole span
                price_selector = './/span[@class="sx-price-whole"]/text()'
                # get product name from data-attribute of h2 tag
                name_selector = './/h2/@data-attribute'
                # get address and/or id from first a tag
                address_selector = './/a/@href'
                id_selector = './/a/@id'
    
                # for each item, yield the following information
                yield {
                    'name': listings.xpath(name_selector).extract_first().strip(),
                    'price': listings.xpath(price_selector).extract_first().strip(),
                    'address': check_address(listings, address_selector, id_selector),
                    'category': category,
                    'scan_date': date}
            
            except:
                # if an error, just write a null entry for easy data cleanup
                yield {
                    'name': listings,
                    'price': 'null',
                    'address': 'null',
                    'category_code': 'null',
                    'scan_date': 'null'}

            # select the next page, which is in the href after the a tag with the title 'Next Page'
            next_page_selector = './/a[@title="Next Page"]/@href'
            next_page = response.xpath(next_page_selector).extract_first()
            # if there is a next page, call it as the next page and recall (Scrapy spider uses recursion)
            if next_page:
                yield scrapy.Request(
                    response.urljoin(next_page),
                    callback=self.parse)

# to run spider by running the script as a python file
# from scrapy common practices, https://doc.scrapy.org/en/latest/topics/practices.html
# COMMENTED OUT SO IT IS NOT RUN IN SCRAPY NOTEBOOK
# if __name__ == "__main__":
#     # start crawler from with command line arguments save_location and logging
#     process = CrawlerProcess({
#             'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
#             'FEED_FORMAT': 'csv',
#             'FEED_URI': sys.argv[1],
#             'LOG_ENABLED': sys.argv[2]
#         })
#     # start spiders crawling
#     process.crawl(AmazonSpider)
#     process.start()

The Amazon scraper was run with terminal command 'python scraper_amazon.py "/Users/josh/Desktop/Machine Learning Research Project/results/amazon_photo.csv'. Here is the data saved imported as a pandas dataframe:

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
# import as latin1 encoding because of characters in name which arent't supprted by utf-8.
amazon = pd.read_csv('results/amazon_photo_video.csv', low_memory=False,encoding='latin1')
amazon.head()

,name,price,address,category_code,scan_date
0,Breakthrough Photography 77mm X4 6-Stop ND Fil...,173,https://www.amazon.com/Breakthrough-Photograph...,Camera & Photo: Electronics,2018-12-11
1,Canon VIXIA HF R800 57X Zoom Full HD 1080P Vid...,279,https://www.amazon.com/R800-Camcorder-Tripod-D...,Camera & Photo: Electronics,2018-12-11
2,Nikon AF-S FX NIKKOR 50mm f/1.8G Lens with Aut...,216,https://www.amazon.com/Nikon-AF-S-NIKKOR-1-8G-...,Camera & Photo: Electronics,2018-12-11
3,Canon EF 100mm f/2.8L IS USM Macro Lens for Ca...,699,https://www.amazon.com/Canon-100mm-Macro-Digit...,Camera & Photo: Electronics,2018-12-11
4,Sony - E 50mm F1.8 OSS Portrait Lens (SEL50F18/B),273,https://www.amazon.com/Sony-50mm-F1-8-Portrait...,Camera & Photo: Electronics,2018-12-11


### Creating Kijiji and Craigslist Spiders
The core logic if these spiders remain the same as the Amazon scraper, so I will not go into their detail. They are made to obtain the item name, price, address, city, category_code, city_code, business, and scan_date from Kijiji and Craigslist. 

However, since these scrapers would need to be run daily their coding practices are improved and the scrapers themselves. Instead of simply initializing the standard scrapy Spider, these spiders overwrite the regular scapy initialization to add custom paramets like city, start_urls, and more to the spider. This way, a master scrape file can be used to run multiple spiders from one command.

In addition to collecitng the raw data from the webpage, the scrapers have a variety of functions to clean the raw data. The functions relevant to the ML feature selection will be brriefly explained in the machine learning section of this report.

Then, a master scrape file is used to run and manage all the spiders. It runs based on the settings from a JSON config file, and the data is uploaded to a local database. This database stores the scanning profiles to send to the spiders, urls to scan, etc. The various unique raw category and city codes are consolidated into one uniform list for consitency across data sources. To reiterate, the core logic of the scrapers is the exact same as the Amazon scraper. However, there is a large amount of code to optimize the workflow and recurring scanning since an arbitrage solution would need to be modulable and run frequently. For the purpose of this report, the scan data will be added from the csvs outputted from the scraper and not the data written to the database.

Explaining the syntax and technical implementations of the database and master scrape file is out of scope for this report. However, the code is here for those interested. If one creates a local mysql database with the same structure as listed in the config file, the code should work as the master scraper file is also capable of creating tables for the database. However, this has not been tested on another machine.

Here is the code for the Kijiji Scraper:

In [3]:
import scrapy
import sys
import time
from scrapy.crawler import CrawlerProcess


class KijijiSpider(scrapy.Spider):
    name = "kijiji_spider"

    def __init__(self, *a, **kw):
        super(KijijiSpider, self).__init__(*a, **kw)
        self.start_urls = kw.get("start_urls")
        self.city = kw.get("city")
        self.time = time.strftime('%Y-%m-%d %H:%M:%S')
        self.allowed_domains = ["kijiji.ca"]

    def parse(self, response):
        info_selector = '.info-container'
        for listings in response.css(info_selector):
            price_selector = './/div[@class="price"]/text()'
            name_selector = 'a ::text'
            address_selector = './/div[@class="title"]/a/@href'
            raw_url = 'http://www.kijiji.ca' + listings.xpath(address_selector).extract_first()

            yield {
                'name': listings.css(name_selector).extract_first().strip(),
                'price': self.clean_price(listings.xpath(price_selector).extract_first().strip()),
                'address': self.trim_url(raw_url),
                'city': self.city,
                'category_code': self.get_category_code(raw_url),
                'city_code': self.get_city_code(raw_url),
                'business': self.is_business(raw_url),
                'scan_date': self.time
            }

            next_page_selector = './/span[@title="Next"]/@data-href'
            next_page = 'http://www.kijiji.ca' + response.xpath(next_page_selector).extract_first()
            if next_page:
                yield scrapy.Request(
                    response.urljoin(next_page),
                    callback=self.parse
                )

    @staticmethod
    def clean_price(price):
        # if possible, remove the $, commas, and convert price to a float
        price_strings = ["Please Contact", "Free", "Swap / Trade"]
        if price in price_strings:
            return "Invalid"
        else:
            price = price.replace(",", "")
            return float(price[1:])

    @staticmethod
    def __after_nth_substring(string, substring, n):
        # return the string after nth occurrence of the substring
        string_trim = string
        for i in range(0, n):
            string_trim = string_trim[string_trim.find(substring) + 1:]
        return string_trim

    @staticmethod
    def is_business(url):
        # the listing is marked as a business posting if there is a c
        # before the unique number which is after the 4th /
        if KijijiSpider.__after_nth_substring(url, '/', 6)[0] == 'c':
            return 1
        else:
            return 0

    @staticmethod
    def get_city_code(url):
        # returns the city code which is stored between the 4th and 5th /
        url = KijijiSpider.__after_nth_substring(url, '/', 4)
        return url[:url.index('/')]

    @staticmethod
    def get_category_code(url):
        # returns the city code which is stored between the 4th and 5th /
        url = KijijiSpider.__after_nth_substring(url, '/', 3)
        return url[:url.index('/')]

    @staticmethod
    def trim_url(url):
        # returns only the information required for the url to work
        if '?' in url:
            return url[:url.index('?')]
        else:
            return url


class Main:
    @staticmethod
    def run(**kwargs):
        # start the crawler with the provided logging toggle, save path, and city name
        process = CrawlerProcess({
            'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
            'FEED_FORMAT': 'csv',
            'LOG_ENABLED': kwargs.get('logging')
        })
        Main.add_crawler(process, path=kwargs.get('path'), start_urls=[kwargs.get('start_urls')],
                         city=[kwargs.get('city')])
        process.start()

    @staticmethod
    def add_crawler(crawler_process, **kwargs):
        # adds the spider to the crawler process by adding custom spider to process
        # creating custom spider class based off of original spider to override the custom_settings
        # class variable which is instantiated before the __init__ of the original spider class
        # this hacky fix is required to run multiple spiders with different feed exports
        custom_spider = type("custom_spider", (KijijiSpider,),
                             {"custom_settings": {'FEED_URI': kwargs.get('path')}})
        crawler_process.crawl(custom_spider, start_urls=kwargs.get('start_urls'),
                              city=kwargs.get('city'))

# COMMENTED OUT SO IT IS NOT RUN IN SCRAPY NOTEBOOK
# if __name__ == "__main__":
#     # start crawler from command line arguments for testing
#     Main.run(path=sys.argv[1], logging=sys.argv[3],
#              city=sys.argv[4], start_urls=sys.argv[2])

Here is the code for the craigslist scraper:

In [4]:
import scrapy
import sys
import time
from scrapy.crawler import CrawlerProcess


class CraigslistSpider(scrapy.Spider):
    name = "craigslist_spider"

    def __init__(self, *a, **kw):
        super(CraigslistSpider, self).__init__(*a, **kw)
        self.start_urls = kw.get("start_urls")
        self.city = kw.get("city")
        self.time = time.strftime('%Y-%m-%d %H:%M:%S')
        self.allowed_domains = ["craigslist.ca", "craigslist.com"]

    def parse(self, response):
        item_selector = '//p[@class="result-info"]'
        for listings in response.xpath(item_selector):
            price_selector = ".//span[@class='result-price']/text()"
            name_selector = 'a ::text'
            address_selector = 'a/@href'
            url = listings.xpath(address_selector).extract_first()

            yield {
                'name': listings.css(name_selector).extract_first().strip(),
                'price': self.clean_price(listings.xpath(price_selector).extract_first()),
                'address': url,
                'city': self.city,
                'category_code': self.get_category_code(url),
                'city_code': self.get_city_code(url),
                'business': self.is_business(url),
                'scan_date': self.time
            }

            next_page_selector = './/link[@rel="next"]/@href'
            next_page = response.xpath(next_page_selector).extract_first()
        if next_page:
            yield scrapy.Request(
                response.urljoin(next_page),
                callback=self.parse
            )

    @staticmethod
    def __after_nth_substring(string, substring, n):
        # return the string after nth occurrence of the substring
        string_trim = string
        for i in range(0, n):
            string_trim = string_trim[string_trim.find(substring) + 1:]
        return string_trim

    @staticmethod
    def clean_price(price):
        # if possible, remove the $, commas, and convert price to a float
        if price is None:
            return "Invalid"
        else:
            price = price.replace(",", "")
            return float(price[1:])

    @staticmethod
    def get_city_code(url):
        # returns the city code which is stored between the 3rd and 4th /
        url = CraigslistSpider.__after_nth_substring(url, '/', 3)
        return url[:url.index('/')]

    @staticmethod
    def is_business(url):
        # the listing is marked as a business posting if the poster identifier's
        # 3rd character is a d. Identifier is stored after the 4th /
        if CraigslistSpider.__after_nth_substring(url, '/', 4)[2] == 'd':
            return 1
        else:
            return 0

    @staticmethod
    def get_category_code(url):
        # returns the city code which is stored between the 4th and 5th /
        url = CraigslistSpider.__after_nth_substring(url, '/', 4)
        return url[:url.index('/')]


class Main:
    @staticmethod
    def run(**kwargs):
        process = CrawlerProcess({
            'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
            'FEED_FORMAT': 'csv',
            'LOG_ENABLED': kwargs.get('logging'),
            'DOWNLOAD_DELAY': '2',
            'RANDOMIZE_DOWNLOAD_DELAY': '2'
        })
        Main.add_crawler(process, path=kwargs.get('path'), start_urls=[kwargs.get('start_urls')],
                         city=kwargs.get('city'))
        process.start()

    @staticmethod
    def add_crawler(crawler_process, **kwargs):
        # adds the spider to the crawler process by adding custom spider to process
        # creating custom spider class based off of original spider to override the custom_settings
        # class variable which is instantiated before the __init__ of the original spider class
        # this hacky fix is required to run multiple spiders with different feed exports
        custom_spider = type("custom_spider", (CraigslistSpider,),
                             {"custom_settings": {'FEED_URI': kwargs.get('path')}})
        crawler_process.crawl(custom_spider, start_urls=kwargs.get('start_urls'),
                              city=kwargs.get('city'))

# COMMENTED OUT SO IT IS NOT RUN IN SCRAPY NOTEBOOK
# if __name__ == "__main__":
#     Main.run(path=sys.argv[1], logging=sys.argv[3],
#              city=sys.argv[4], start_urls=sys.argv[2])

Here is the code for the scraper config file:

Here is the code for the master scrape file:

In [5]:
import scrapers.scraper_craigslist as Craigslist
import scrapers.scraper_kijiji as Kijiji
from scrapy.crawler import CrawlerProcess
# import pymysql
import datetime
import json


class ScanDBManager:
    def __init__(self, db_set, basic_set):
        try:
            self.db = pymysql.connect(host=db_set['host'],
                                      user=db_set['user'],
                                      passwd=db_set['password'],
                                      db=db_set['database'],
                                      charset="utf8")
        except Exception as e:
            raise ConnectionError("Could not connect to the database because of error: %s \n" % e.__str__())
            return
        self.cursor = self.db.cursor()
        self.set_unicode()
        self.word_filter = basic_set['word_filter']
        self.char_filter = basic_set['char_filter']
        try:
            self.check_tables(db_set['scan_table'], db_set['save_table'])
        except (NotADirectoryError, UserWarning) as e:
            print("Error: %s \n has prevented a proper database connection" % e.__str__())

        self.scan_table = db_set['scan_table']
        self.save_table = db_set['save_table']

    def set_unicode(self):
        self.cursor.execute('SET NAMES utf8;')
        self.cursor.execute('SET CHARACTER SET utf8;')
        self.cursor.execute('SET character_set_connection=utf8;')

    def check_tables(self, scan_table, save_table):
        # check if there is a scan table
        if self.cursor.execute("SHOW TABLES LIKE '%s';" % scan_table) == 0:
            raise NotADirectoryError("There is no scan table called '%s")

        # check if the scan table has any links
        if self.cursor.execute("SELECT * FROM %s;" % scan_table) == 0:
            raise UserWarning("There are entries in table '%s' to scan" % scan_table)

        # check if there is a results table, if not create one
        if self.cursor.execute("SHOW TABLES LIKE '%s';" % save_table) == 0:
            self.create_results_table(save_table)

    def create_results_table(self, table_name):
        # create a table to save the results. This should not be hardcoded
        # but move to a config file before launch
        query = """
                CREATE TABLE %s (
                  name VARCHAR(255) NOT NULL,
                  clean_name VARCHAR(255),
                  price INT NOT NULL,
                  address VARCHAR(255) PRIMARY KEY NOT NULL,
                  date DATE NOT NULL,
                  category_code VARCHAR(255) NOT NULL,
                  category VARCHAR(255),
                  city_code VARCHAR(255),
                  city VARCHAR(255),
                  description TEXT,
                  comment TEXT,
                  uploader VARCHAR(255),
                  duplicate BOOLEAN DEFAULT FALSE,
                  profitable BOOLEAN,
                  last_updated DATETIME,
                  active BOOLEAN
                );
                """"" % table_name
        self.cursor.execute(query)

    def get_to_scan(self):
        # returns a list of all the links to scan
        self.cursor.execute("SELECT * FROM %s;" % self.scan_table)
        to_scan = self.cursor.fetchall()
        return to_scan

    def get_sources(self):
        self.cursor.execute("SELECT DISTINCT source FROM %s;" % self.scan_table)
        sources = []
        for source in self.cursor.fetchall():
            sources.append(source[0])
        return sources

    def write_scan(self, data):
        query = "INSERT INTO scan_results (name, clean_name, price, address, date, " \
        "category_code, city, last_updated, active) VALUES " \
        "('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s');" % (
            data[0], self.clean_name(data[0], self.char_filter, self.word_filter),
            data[1], data[2], data[3], data[4], data[5], data[6], data[7])
        try:
            self.cursor.execute(query)
            self.db.commit()
        except pymysql.Error as e:
            print("Error '%s' as a result of query '%s'" %(e.__str__(), query))

    def clean_name(self, raw_name, char_filter, word_filter):
        # removes any chars from the char filter and removes words from the word filter
        # returns in lower case
        try:
            raw_name = raw_name.lower()
            for c in char_filter:
                raw_name = raw_name.replace(c, " ")
            raw_name = ' '.join(raw_name.split())
            words = raw_name.split(' ')
            clean_words = []
            for word in words:
                if (word not in word_filter) and ('$' not in word) and (len(word) > 1):
                    clean_words.append(word)
            name = " ".join(clean_words)
        except Exception as e:
            print("Error cleaning name %s because of exception: %s" % e.__str__)
            name = raw_name
        return name

    def check_primary(self, table, column, key):
        return self.cursor.execute("SELECT 1 FROM %s WHERE %s = '%s' LIMIT 1" % (table, column, key))

    def close(self):
        self.db.close()


class ScanManager:
    @staticmethod
    def scan(process, to_scan, scan_process):
        for link in to_scan:
            if link[2] in scan_process:
                scan_profile = scan_process.get(link[2])
                spider = scan_profile[1]
                path = scan_profile[0]
                spider.add_crawler(process, path=path,
                                   start_urls=[link[1]], city=link[0])
            else:
                raise UserWarning("There is no scan process for source '%s" % link[2])

    @staticmethod
    def main(settings):
        try:
            db = ScanDBManager(settings["LOCAL"], settings["BASIC"])
        except ConnectionError as e:
            print(e)
        process = CrawlerProcess(settings["CRAWLER"])
        to_scan = db.get_to_scan()
        scan_profiles = ScanManager.generate_profiles(settings["SCAN"])
        ScanManager.scan(process, to_scan, scan_profiles)
        process.start()

    @staticmethod
    def generate_profiles(profiles):
        scan_profiles = {}
        start_time = datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")
        for profile in profiles:
            data = []
            for param in profiles.get(profile):
                if (param[-3:]) == "csv":
                    data.append(param[:-4] + " " + start_time + ".csv")
                elif param[-6:] == "Main()":
                    try:
                        data.append(eval(param))
                    except NameError:
                        print("The scraper class %s could not be read. Make sure you are importing a scraper "
                              "with the proper name" % param)
                else:
                    raise ValueError("Unknown data type '%s' in profile %s" % (param, profile))
            scan_profiles[profile] = data
        return scan_profiles

# COMMENTED OUT SO IT IS NOT RUN IN SCRAPY NOTEBOOK
# if __name__ == '__main__':
#     with open('config.json', 'r') as f:
#         config = json.load(f)
#     ScanManager.main(config)

Python script to upload results to database:

In [6]:
import csv
#import pymysql
import os
#import scrape
import json


class CSVManager:
    def __init__(self, **kwargs):
        self.path = kwargs.get('path')
        self.sources = kwargs.get('sources')
        if not self.get_csvs():
            raise NotADirectoryError("There is no scan scan data in directory '%s" % self.path)
        self.csvs = self.get_csvs()

    def get_csvs(self):
        # get all the csv from the results directory
        csvs = []
        for file in os.listdir(self.path):
            if file[:file.find(" ")] in self.sources:
                csvs.append(self.path + "/" + file)
        return csvs

    def write_csvs(self, db):
        # writes data from every csv
        for path in self.csvs:
            self.write_data(path, db)

    @staticmethod
    def write_data(path, db):
        def check_valid(data):
            # checks to see if price is valid and that there is no item with the same url in the db
            if data[1] is not "Invalid" and data[6] == '0':
                if db.check_primary(db.save_table, "address", data[2]) == 0:
                    return True
                else:
                    return False

        # writes all the data
        with open(path) as f:
            reader = csv.reader(f)
            for row in reader:
                if check_valid(row) is True:
                    db.write_scan([row[0].replace("'", "").encode('ascii', 'replace').decode(),
                                   int(row[1][:row[1].find('.')]), row[2], row[7][:row[7].find(' ')],
                                   row[4], row[3], row[7], "1"])

    def move_scans(self):
        # puts all the csv files from the results directory to the old subdirectory
        # which contains a folder for each source
        for path in self.csvs:
            filename = self.__after_nth_substring(path, '/', 6)
            source = filename[:filename.index(' ')]
            subdirectory = path[:path.index(filename)] + "old/" + source + "/"
            if not os.path.exists(subdirectory):
                os.makedirs(subdirectory)
            new_path = subdirectory + filename
            os.rename(path, new_path)

    @staticmethod
    def __after_nth_substring(string, substring, n):
        # return the string after nth occurrence of the substring
        string_trim = string
        for i in range(0, n):
            string_trim = string_trim[string_trim.find(substring) + 1:]
        return string_trim

# COMMENTED OUT SO IT IS NOT RUN IN SCRAPY NOTEBOOK
# if __name__ == "__main__":
#     # writes all the valid items from the scan csvs to the server
#     with open('config.json', 'r') as f:
#         config = json.load(f)
#     results_db = scrape.ScanDBManager(config["LOCAL"], config["BASIC"])
#     scan_sources = results_db.get_sources()
#     manager = CSVManager(path=config["BASIC"]["result_path"],
#                          sources=scan_sources)
#     manager.write_csvs(results_db)
#     manager.move_scans()

Code to export results from database to csv:

Here is the scraped Kijiji and Craigslist data as a pandas dataframe:

In [7]:
kij_craig = pd.read_csv('results/kij_craig.csv', low_memory=False,encoding='latin1')
kij_craig.head()

,clean_name,name,price,address,city,category_code,city_code,business,scan_date
0,minolta 300si film camera bundle 35-70mm f4 70...,Minolta 300si Film Camera Bundle | 35-70mm F4 ...,150.0,https://toronto.craigslist.ca/tor/pho/d/minolt...,Toronto,pho,tor,0,2018-12-09 13:11
1,olympus magellan 8x25 wp binocular,Olympus Magellan 8x25 WP I Binocular,100.0,https://toronto.craigslist.ca/tor/pho/d/olympu...,Toronto,pho,tor,0,2018-12-09 13:11
2,olympus pathfinder exps-1 8x42 porro prism bin...,Olympus Pathfinder EXPS-1 8x42 Porro Prism Bin...,275.0,https://toronto.craigslist.ca/tor/pho/d/olympu...,Toronto,pho,tor,0,2018-12-09 13:11
3,bronica sq-a medium format 6x6,Bronica SQ-A Medium Format 6x6,700.0,https://toronto.craigslist.ca/tor/pho/d/bronic...,Toronto,pho,tor,0,2018-12-09 13:11
4,canon a35f 35mm film rangefinder camera,Canon A35F 35mm Film Rangefinder Camera,150.0,https://toronto.craigslist.ca/tor/pho/d/canon-...,Toronto,pho,tor,0,2018-12-09 13:11


### Record Linkage with ML

### Preprocessing

In the scraping portion of the process, some preprocessing and feature selection was done. Since the relevant data to scrape must be chosen during the development of the spiders, this process is mostly done beforehand. The more features the better, but extracting features is expensive and introduces more noise. So, instead of developing spiders to to get as much metadata as possible to prune during the ML design portion, only sensible data was obtained to start with.

The relevant data is the item name, price, address (used for reference), city, category, city_code, business (binary, 0 or 1, whether or not is being sold by a business, and scan_data. The name and price is relevant so they will be included in training. The business filter is used to filter out results from the getgo since postings by businesses are unlikely to be profitable (since businesses are more likely to price their items properly). However, it will still be included in the model since listing posted by businesses have different naming conventions than an average user. Also, the machine learning model can be used to test this hypotheses. The other data is relevant for the logistics and management of the overall arbitrage solution, but the not the machine learning algorithm component.

Also, some decisions were made during the scraping phase which reduce the effectiveness of the machine learning algorithm but increase its practicality. A minimum price threshold of `$`30 or `$`50 was used during the scraping of Amazon, Craigslist and Kijiji. Items with prices lower than this are often mislabelled on purpose (like at `$`1 to attract buyers). Even if they are accurately named, they are most likely worthwhile arbitrage opportunities since a 10 percent profit margin on a `$`20 item is much less significant compared to the profit margin on a `$`200 item. This means that the model will be innappropriate for matching all items. It is not intended to be an absolutely effective record linkage tool for all products, only a practical solution for linkage of products likely to be profitable arbitrage opportunities. If it only matches well with a minority of total products, but those products represent the majority of arbitrage opportunities, then the ML record linkage algorithm is a success.

Due to time constraints, scraping all of the electronics data from Amazon isn't feasable. So, only one sub category of electronics, photo/video, will be scraped from Amazon. Thus, the data from Craigslist and Kijiji will be filtered to only include items of that cateogory. Since a complete solution would still use an initial category filter before matching, this ML solution is a good representation of the larger possible system.

Cleaning up the x data:

In [8]:
import numpy as np

# filter only results that are in the photo/video category
category_filter = (np.array(kij_craig.category_code == 'pho') | 
                   np.array(kij_craig.category_code == 'phd') | 
                   np.array(kij_craig.category_code == 'v-camera-camcorder-lens'))
x_df = kij_craig[category_filter].copy()

# remove unecessary columns
x_df.drop(x_df.columns[[4,5,6,8]], axis=1, inplace=True)
x_df.head()

,clean_name,name,price,address,business
0,minolta 300si film camera bundle 35-70mm f4 70...,Minolta 300si Film Camera Bundle | 35-70mm F4 ...,150.0,https://toronto.craigslist.ca/tor/pho/d/minolt...,0
1,olympus magellan 8x25 wp binocular,Olympus Magellan 8x25 WP I Binocular,100.0,https://toronto.craigslist.ca/tor/pho/d/olympu...,0
2,olympus pathfinder exps-1 8x42 porro prism bin...,Olympus Pathfinder EXPS-1 8x42 Porro Prism Bin...,275.0,https://toronto.craigslist.ca/tor/pho/d/olympu...,0
3,bronica sq-a medium format 6x6,Bronica SQ-A Medium Format 6x6,700.0,https://toronto.craigslist.ca/tor/pho/d/bronic...,0
4,canon a35f 35mm film rangefinder camera,Canon A35F 35mm Film Rangefinder Camera,150.0,https://toronto.craigslist.ca/tor/pho/d/canon-...,0


The y data format should be as similar to the x data as possible to reduce noise and provide the most accurate labels (to be explained later). The preprocessing required here is relatively simply, just set all the text to lower case and remove non utf-8 supported characters. Unecessary columns were removed as well.

Cleaned up y data:

In [11]:
y_df = amazon.copy()
# remove unecessary columns
y_df.drop(y_df.columns[[3, 4]], axis=1, inplace=True)
# clean name
y_df.name = y_df.name.apply(lambda x: bytes(x, 'utf-8').decode('utf-8', 'ignore').lower())
y_df.head()

,name,price,address
0,breakthrough photography 77mm x4 6-stop nd fil...,173,https://www.amazon.com/Breakthrough-Photograph...
1,canon vixia hf r800 57x zoom full hd 1080p vid...,279,https://www.amazon.com/R800-Camcorder-Tripod-D...
2,nikon af-s fx nikkor 50mm f/1.8g lens with aut...,216,https://www.amazon.com/Nikon-AF-S-NIKKOR-1-8G-...
3,canon ef 100mm f/2.8l is usm macro lens for ca...,699,https://www.amazon.com/Canon-100mm-Macro-Digit...
4,sony - e 50mm f1.8 oss portrait lens (sel50f18/b),273,https://www.amazon.com/Sony-50mm-F1-8-Portrait...


### Labelling

Labels are required for the data since this is a classification problem. A label representing the corresponding product in the Amazon dataset. Clustering could be used and then the clusters be matced with the Amazon dataset, but this is time consuming, innacurate, and poses many challenges since not all products are guaranteed to be matched in the Amazon dataset. Labels could (and should) have be done manually beforehand by an individual. However, due to time constraints this will not be done. It must be acknowledged that this skews the effectiveness of the machine learning algorithm since its labels are only as accurate as the algorithm used to generate them. In this case, that is using Levenshtein Distance for approximate string matching.

Levenshtein distance (LD) is an algorithm to calculate the similarity between two strings. The distance is the number of deletions, insertions, or substitutions required to transform one string to another. The lower the distance, the more similar the strings are (and in this case, the more probable that they are a match). It is widely used in government agencies, and software products to match mispelled or similar strings. In this case, it will be used to match the name of the product from the scraped data from Kijiji and Craigslist to a product name in the Amazon dataset. Now, if there is no similar. Amazon is the best data source given the constraints and significant preprocessing was done. The python implementation of the LD algorithm used is provided by SeatGeek, https://github.com/seatgeek/fuzzywuzzy.

In [13]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

def generate_labels(x_df, y_df):
    # get possible product labels from y dataset
    possible_labels = y_df['name'].unique()
    start_time = time.process_time()
    
    # loop through rows, writing the best possible label along with its LD score
    for i, row in x_df.iterrows():
        try:
            possibilities = process.extract(row['clean_name'], possible_labels, scorer=fuzz.partial_ratio)
            x_df.set_value(i,'fuzz_score', possibilities[0][1])
            x_df.set_value(i,'label', possibilities[0][0])
        except:
            x_df.set_value(i,'fuzz_score', 0)
            x_df.set_value(i,'label', 'error')
            
    # return time it took to generate labels
    elapsed_time = time.process_time() - start_time
    print("It took %i seconds to generate the labels using LD" %elapsed_time)

generate_labels(x_df, y_df)

/Users/josh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/josh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


It took 3717 seconds to generate the labels using LD


### Feature Selection
During the scraping process of Kijiji and Craigslist, the strings were cleaned. This was dony be removing odd characters like brackets and exclamation marks. Unecessary words that pertain to marketing but not the product themselves, like 'Perfect condition' or 'Amazing deal', were also removed using a string parser and word filter which is in the source code above.

The critical feature, the name, need to be turned into a data interpratable by the machine learing algorithm. To do this, a bag-of-words model will be used. This is when each unique word is a feature (token) and its weight is based on the frequency of said word disregarding grammer and order. There are various bag-of-words models which use different patterns to decide the weight of each feature. The TFIDF, term frequency-inverse document frequency, model will be used. This provides more weight to terms that are less frequent in the entire dataset. This way, common occuring words like colors provide little impact but uncommon words like unique model codes do. The sklearn TfidfVectorizer module will be used to generate the vectors.

The extracted words features were not fit the to the training data and then used to transform the testing set, fit and and transformed to the entire data set. Although this is goes against sound ML methadology since it is being in some sense trained to the test set, it is necessary because of the large number of unique labels and relatively small dataset. If an item is in the test set, it is most likely that a different (or same) representation of the same item will be found in the training set. Thus, there would be no dummy variables for the item in the test set to match with. If the dataset was scraped over a long period of time, there would be enough repitition of items to make fitting the text features to just the training set feasible. It is recommend to repeat this process when more data is collected.

Here is the code adding the bag-of-words features with TFIDF:

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

#fuck_me = x_df.copy()
# Vectorizer code based off of: https://stackoverflow.com/questions/45961747/append-tfidf-to-pandas-dataframe

# x_df.name = x_df.clean_name.apply(lambda x: bytes(x, 'utf-8').decode('utf-8', 'ignore').lower())
# x_df.name = x_df.name.apply(lambda x: bytes(x, 'utf-8').decode('utf-8', 'ignore').lower())
# create vectorizer
vectorizer = TfidfVectorizer()
# generate tokens
vector_tokens = vectorizer.fit_transform(x_df['clean_name'])
# create feature df
feature_df = pd.DataFrame(vector_tokens.toarray(), columns=vectorizer.get_feature_names())
# merge feature df to the original df
x_df_featured = pd.concat([x_df, feature_df], axis=1).copy()
# display results
x_df_featured.iloc[range(0,5)]

clean_name  \
0  minolta 300si film camera bundle 35-70mm f4 70...   
1                 olympus magellan 8x25 wp binocular   
2  olympus pathfinder exps-1 8x42 porro prism bin...   
3                     bronica sq-a medium format 6x6   
4            canon a35f 35mm film rangefinder camera   

                                                name  price  \
0  Minolta 300si Film Camera Bundle | 35-70mm F4 ...  150.0   
1               Olympus Magellan 8x25 WP I Binocular  100.0   
2  Olympus Pathfinder EXPS-1 8x42 Porro Prism Bin...  275.0   
3                     Bronica SQ-A Medium Format 6x6  700.0   
4            Canon A35F 35mm Film Rangefinder Camera  150.0   

                                             address  business  fuzz_score  \
0  https://toronto.craigslist.ca/tor/pho/d/minolt...       0.0        60.0   
1  https://toronto.craigslist.ca/tor/pho/d/olympu...       0.0        63.0   
2  https://toronto.craigslist.ca/tor/pho/d/olympu...       0.0        69.0   
3  https://toronto.craigslist.ca/tor/pho/d/bronic...       0.0        63.0   
4  https://toronto.craigslist.ca/tor/pho/d/canon-...       0.0        72.0   

                                               label   00   01  025  028  \
0                          moultrie m-50 game camera  0.0  0.0  0.0  0.0   
1                         steiner xc 8x42 binoculars  0.0  0.0  0.0  0.0   
2  fujinon mariner 7x50 wpc-xl porro prism binocular  0.0  0.0  0.0  0.0   
3  hasselblad h6d-50c medium format dslr camera, ...  0.0  0.0  0.0  0.0   
4  leica mp 10302 35mm rangefinder camera with 0....  0.0  0.0  0.0  0.0   

   028b  029  029mk2   04  0509  055  055b  055c  055mf3  055xprob  057   0l  \
0   0.0  0.0     0.0  0.0   0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0   
1   0.0  0.0     0.0  0.0   0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0   
2   0.0  0.0     0.0  0.0   0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0   
3   0.0  0.0     0.0  0.0   0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0   
4   0.0  0.0     0.0  0.0   0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0   

   0mp   0t   0x   10  100  1000  1000d  1000fda  1000s  1000ss  100633  \
0  0.0  0.0  0.0  0.0  0.0   0.0    0.0      0.0    0.0     0.0     0.0   
1  0.0  0.0  0.0  0.0  0.0   0.0    0.0      0.0    0.0     0.0     0.0   
2  0.0  0.0  0.0  0.0  0.0   0.0    0.0      0.0    0.0     0.0     0.0   
3  0.0  0.0  0.0  0.0  0.0   0.0    0.0      0.0    0.0     0.0     0.0   
4  0.0  0.0  0.0  0.0  0.0   0.0    0.0      0.0    0.0     0.0     0.0   

   100iso  100m  100mm  100ws  100x  101  102  1024046  102aw  104a09nii  105  \
0     0.0   0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   
1     0.0   0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   
2     0.0   0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   
3     0.0   0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   
4     0.0   0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   

   105mm  105vr  1065  1066x  107_  1080  1080p  10d  10feet  10l  10m  10mb  \
0    0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   0.0   
1    0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   0.0   
2    0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   0.0   
3    0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   0.0   
4    0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   0.0   

   10mm  10mp  10x  10x10   11  110  110mm  110v  1150w  116  118x   12  120  \
0   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0  0.0  0.0   
1   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0  0.0  0.0   
2   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0  0.0  0.0   
3   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0  0.0  0.0   
4   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0  0.0  0.0   

   1200  1200d  120

### Splitting the data for training and testing
 

Training the model on inacurately labled data is a recipe for disaster. Since the labels were generated with an algorithm, extra percaution is required. To ensure that only the most accurately labled data is used, the x dataframe will only include data which with a high scoring metric (as scored by fuzzywuzzy).

Then, it will be randomly split into training with an 80/20 split. 

The code to do so is as follows:

In [56]:
from sklearn.model_selection import train_test_split

# seperates the data into x and y
def seperate_features():
    # removing null columns. Not sure they are present, possibly because of encoding conversion problem
    # or because of error in feature vectorizer. Regardless, not enough time to fix.
    error_map=pd.isnull(x_df_featured).any(axis=1)
    x_df_featured_clean = x_df_featured[np.invert(error_map)]

    # filter only well labled results
    x_data = x_df_featured_clean[x_df_featured_clean.fuzz_score > 80].copy()
    # seperate labels
    y_data = x_data['label']
    x_data.drop(['label'], axis=1, inplace=True)
    # create feature list
    feature_set = [x for x in list(x_data) if x not in ['clean_name', 'name', 'address', 'fuzz_score']]

# split the data into sets for training and testing    
X_train, X_test, y_train, y_test = train_test_split(x_data[feature_set], y_data, test_size = 0.2, random_state = 4)

print("Training dimensions: %s" %str(X_train.shape))
print("Testing dimensions: %s" %str(X_test.shape))
print("Testing data y: ")
print(y_train[0:4])
print("Testing data x: ")
X_train.iloc[range(0,4)]


Training dimensions: (496, 3744)
Testing dimensions: (124, 3744)
Testing data y: 
645                          canon mount adapter ef-eos r
806     zeiss 35mm f/2 biogon t* zm mf lens for zeiss ...
3470                                      polaroid camera
80                                         nikon 1 nikkor
Name: label, dtype: object
Testing data x: 


price  price  business   00   01  025  028  028b  029  029mk2   04  \
645   1000.0    0.0       0.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0   
806     80.0    0.0       1.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0   
3470    55.0    0.0       0.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0   
80    1850.0    0.0       0.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0   

      0509  055  055b  055c  055mf3  055xprob  057   0l  0mp   0t   0x  \
645    0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0  0.0  0.0  0.0   
806    0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0  0.0  0.0  0.0   
3470   0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0  0.0  0.0  0.0   
80     0.0  0.0   0.0   0.0     0.0       0.0  0.0  0.0  0.0  0.0  0.0   

            10  100  1000  1000d  1000fda  1000s  1000ss  100633  100iso  \
645   0.000000  0.0   0.0    0.0      0.0    0.0     0.0     0.0     0.0   
806   0.000000  0.0   0.0    0.0      0.0    0.0     0.0     0.0     0.0   
3470  0.628466  0.0   0.0    0.0      0.0    0.0     0.0     0.0     0.0   
80    0.000000  0.0   0.0    0.0      0.0    0.0     0.0     0.0     0.0   

      100m  100mm  100ws  100x  101  102  1024046  102aw  104a09nii  105  \
645    0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   
806    0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   
3470   0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   
80     0.0    0.0    0.0   0.0  0.0  0.0      0.0    0.0        0.0  0.0   

      105mm  105vr  1065  1066x  107_  1080  1080p  10d  10feet  10l  10m  \
645     0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   
806     0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   
3470    0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   
80      0.0    0.0   0.0    0.0   0.0   0.0    0.0  0.0     0.0  0.0  0.0   

      10mb  10mm  10mp  10x  10x10   11  110  110mm  110v  1150w  116  118x  \
645    0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0   
806    0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0   
3470   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0   
80     0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0   0.0    0.0  0.0   0.0   

       12  120  1200  1200d  120mm  120obo  1228  124  124g  12504  125mm  \
645   0.0  0.0   0.0    0.0    0.0     0.0   0.0  0.0   0.0    0.0    0.0   
806   0.0  0.0   0.0    0.0    0.0     0.0   0.0  0.0   0.0    0.0    0.0   
3470  0.0  0.0   0.0    0.0    0.0     0.0   0.0  0.0   0.0    0.0    0.0   
80    0.0  0.0   0.0    0.0    0.0     0.0   0.0  0.0   0.0    0.0    0.0   

      125w  126  127  128  1280  128gb  12bit  12kg  12mm  12mp  12x  12x50  \
645    0.0  0.0  0.0  0.0   0.0    0.0    0.0   0.0   0.0   0.0  0.0    0.0   
806    0.0  0.0  0.0  0.0   0.0    0.0    0.0   0.0   0.0   0.0  0.0    0.0   
3470   0.0  0.0  0.0  0.0   0.0    0.0    0.0   0.0   0.0   0.0  0.0    0.0   
80     0.0  0.0  0.0  0.0   0.0    0.0    0.0   0.0   0.0   0.0  0.0    0.0   

      12xp   13  1300  1300d  1300mm  1300w  135  135mm  136  137  13inch  \
645    0.0  0.0   0.0    0.0     0.0    0.0  0.0    0.0  0.0  0.0     0.0   
806    0.0  0.0   0.0    0.0     0.0    0.0  0.0    0.0  0.0  0.0     0.0   
3470   0.0  0.0   0.0    0.0     0.0    0.0  0.0    0.0  0.0  0.0     0.0   
80     0.0  0.0   0.0    0.0     0.0    0.0  0.0    0.0  0.0  0.0     0.0   

       14  140  1400  140mm  14100  141rc  145  14e  14ex  14mm  14mp   15  \
645   0.0  0.0   0.0    0.0    0.0    0.0  0.0  0.0   0.0   0.0   0.0  0.0   
806   0.0  0.0   0.0    0.0    0.0    0.0  0.0  0.0   0.0   0.0   0.0  0.0   
3470  0.0  0.0   0.0    0.0    0.0    0.0  0.0  0.0   0.0   0.0   0.0  0.0   
80    0.0  0.0   0.0    0.0    0.0    0.0  0.0  0.0   0.0   0.0   0.0  0.0   

      150  1500  150mm  150watt  1535  1540  1541  156  15784  158  15mm  \
645   0.0   0.0    0.0      0.0   0.0   0.0   0.0  0.0    0.0  0.0   0.

### Training the Model

There are a variety of possible applicable classification models. For small data sizes, linear SVC is a general classfier. For text specific data, naive bayes tends to produce better results. A model was trained for both, in addition to a rbf SVC model for comparison.

Here is the code:

In [70]:
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

# simple rbf classifier
svm_clf = svm.SVC(kernel = 'rbf', C = 100000, gamma = 1.5e-10, 
                  random_state = 103, probability = True)
svm_clf.fit(X_train, y_train)
print("The SVM classifier has a score of %f with the test data" 
      %svm_clf.score(X_test, y_test))
print("The SVM classifier has a score of %f with the training data" 
      %svm_clf.score(X_train, y_train))

# simple linear classifier
linear_clf = LinearSVC(random_state=0, tol=1e-5)
linear_clf.fit(X_train, y_train)
print("The Linear SVC classifier has a score of %f with the test data" 
      %linear_clf.score(X_test, y_test))
print("The Linear SVC classifier has a score of %f with the training data" 
      %linear_clf.score(X_train, y_train))

# simpe gaussian classified
naive_clf = GaussianNB()
naive_clf.fit(X_train, y_train)
print("The naive bayes classifier has a score of %f with the test data" 
      %naive_clf.score(X_test, y_test))
print("The naive bayes classifier has a score of %f with the training data" 
      %naive_clf.score(X_train, y_train))

The SVM classifier has a score of 0.040323 with the test data
The SVM classifier has a score of 0.046371 with the training data
The Linear SVC classifier has a score of 0.000000 with the test data
The Linear SVC classifier has a score of 0.004032 with the training data
The naive bayes classifier has a score of 0.016129 with the test data
The naive bayes classifier has a score of 0.993952 with the training data


Normally, gridsearch would be applied to choose the most optimal parameters for the models. However, all the models produced such terrible results that it isn't worthwhile, with success rates less than 5%. The linear and rbf kernel classifier were incapable of fitting the training or testing data. The naive bayes model was capable of fitting the testing data, but I had so much overfitting that it failed with the test data. 

Feature pruning and dimensionality reduction could greatly improve these models, but is not appropriate since removing features would remove the key and only identifiers for certain items. For example, the sparse 'a7ii' model code would likely not be included, rendering the classifier useless for that item. Dimensionality reduction would lend the model to be accurate in identifying only some common items which is inapropriate for the application of retail arbitrage.

### Summary

In conclusion, the application of Machine Learning for record linkage in this context of retail arbitrage is not an appropriate alternative to pre-existing data matching algorithms. 
1. Having training focused on the features extracted from only one paramer, text
2. Providing only date of scraped items to classify instead of a larger timeline
3. Such a large amount of possible labels for such a small dataset
4. The inherit error of the generated item labels
5. The large amount of noise from the inconsistent user inputted data from Kijiji and Craigslist

Every item would require multiple text related features. So, there is always going to be the curse of dimensionality. Since most of these features are required, the curse of dimensionality will always be present

Although the ML algorithms failed in this context, the exersize was not a failure overall. The data collection and label generation methadology is applicable for other ML scenarious when investing aternative algorithms. For example, if aiming to use ML to replace an expensive brute for algorithm, using said algorithm to predict labels is a good method to replace human labelling. Since many models exists initially to demonstrate proof of concept, the inherit error of labels is acceptable. When training the actual model, perfect human labels would be ideal, but the algorithmic label shortcut is sound for simple proof of concept applications.

ML isn't a complete writeoff in the context of retail arbitrage, but is for identifying product names. If data was missing, like category, ML could be used to generate the missing data since there would be significantly less total category labels than item labels. Once trained, the ML algorithm is quite fast so it is worth investigation if there are data linkage problems with a low count of possible labels. ML could also be applicable in aspects of the data collection and preprocessing, like using dimensionality to clean junk words and characters from item names instead of the bruteforce text pre-processing already in place.